In [6]:
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
import numpy as np


In [31]:
complete_data = pd.ExcelFile('All_Data copy.xls')  # /home/ishangupta/mysite/data/All_Data copy.xls

sheet_names = complete_data.sheet_names  # global

main_df = complete_data.parse(sheet_names[0])

columns = main_df.columns
main_df.columns = ['Mass'] + ['2mm ' + x for x in columns[1:]]

for x in sheet_names[1:]:
    df = complete_data.parse(x)
    columns = df.columns
    df = df.drop('Mass', axis=1)
    df.columns = [x + ' ' + i for i in columns[1:]]
    main_df = main_df.join(df)

del df
columns = main_df.columns[1:]


# print(df.head(3))
# print(energy_values)
def reduce(sheet, threshold):
    transposed = sheet.T
    columns = transposed.columns
    delete = []
    for x in columns:
        if transposed[x][0] > 1000 or sum(transposed[x][1:]) < threshold:
            delete.append(x)
    transposed = transposed.drop(delete, axis=1)
    return transposed.T


# educed_df = reduce(main_df, 48)
# get_discrete works takes in 5 inputs and returns a spin off of the main df that has the clusters specific to the
# molecule ratios / constitution being looked for
def get_discrete(mass_1, number_1, error, mass_2=0, number_2=0):
    def process(x, mass_number, bounds, start=0):
        if (x - start) % mass_number > (mass_number - bounds):
            return ((x - start) % mass_number) - mass_number
        else:
            return (x - start) % mass_number

    mass_start_1 = mass_1 + 1
    mass_end_1 = number_1 * mass_1 + 1
    mass_start_2 = mass_end_1 + mass_2
    mass_end_2 = mass_start_2 + (number_2 - 1) * mass_2
    temp_df = main_df[(main_df['Mass'] < (mass_end_2 + error)) & (main_df['Mass'] > (mass_start_1 - error))]
    array_of_masses = [mass_start_1 + (mass_1 * i) for i in range(number_1)] + [mass_start_2 + (mass_2 * i) for i in
                                                                                range(number_2)]
    # print(array_of_masses)
    yo = [process(x, mass_1, error, start=mass_start_1) if x < (mass_end_1 + error) else process(x, mass_2, error,
                                                                                                 start=mass_start_2) for
          x in temp_df['Mass']]

    # print(temp_df['Mass'])
    # print(yo)
    # print(len(temp_df))
    # print(len(yo))
    yo = [x if error >= x >= -error else 0 for x in yo]
    a = []
    counter = 1
    for x in range(len(yo) - 1):
        if yo[x] != 0:
            a.append(counter)
            if yo[x] != 0 and yo[x + 1] == 0:
                counter = counter + 1
        else:
            a.append(0)
    a.append(counter)
    # print(a)
    # print(temp_df['Mass'])
    temp_df = temp_df.drop(['Mass'], axis=1)
    temp_df['Mass'] = a
    temp_df = temp_df.groupby('Mass').max()
    temp_df = temp_df.drop(labels=[0])
    # print(temp_df.columns)
    temp_df["Peaks_Mass"] = np.ones(len(temp_df)) * np.array(array_of_masses)
    # print(temp_df)
    return temp_df


In [53]:
dataframe = get_discrete(18,50,0.4,0,0)

In [54]:
dataframe

,2mm 10.0,2mm 10.1,2mm 10.2,2mm 10.3,2mm 10.4,2mm 10.5,2mm 10.6,2mm 10.7,2mm 10.8,2mm 10.9,...,35mm 13.7,35mm 13.8,35mm 13.9,35mm 14.0,35mm 14.1,35mm 14.2,35mm 14.3,35mm 14.4,35mm 14.5,Peaks_Mass
Mass,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,1,0,0,1,...,5,1,1,4,3,1,2,2,3,19.0
2,0,0,0,0,0,0,0,0,0,0,...,2,2,1,4,2,1,1,2,3,37.0
3,0,0,0,0,0,0,0,0,0,0,...,2,3,3,3,2,2,2,4,4,55.0
4,0,0,0,1,0,0,0,0,0,1,...,3,3,4,3,6,2,2,3,5,73.0
5,0,0,0,0,0,0,0,0,0,1,...,4,4,3,4,4,5,4,5,10,91.0
6,0,0,0,0,0,0,0,0,0,0,...,5,7,5,7,3,3,3,4,4,109.0
7,0,0,0,0,0,0,1,0,0,0,...,2,6,5,4,2,6,4,5,8,127.0
8,0,0,0,0,0,0,0,0,0,1,...,2,4,5,5,3,3,2,8,7,145.0
9,0,0,0,0,0,0,0,0,0,1,...,3,5,4,2,1,0,3,3,5,163.0


In [55]:
transposed_cluster = dataframe.T
transposed_cluster.columns = dataframe['Peaks_Mass']
transposed_cluster

Peaks_Mass,19.0,37.0,55.0,73.0,91.0,109.0,127.0,145.0,163.0,181.0,...,739.0,757.0,775.0,793.0,811.0,829.0,847.0,865.0,883.0,901.0
2mm 10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2mm 10.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2mm 10.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2mm 10.3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2mm 10.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35mm 14.2,1.0,1.0,2.0,2.0,5.0,3.0,6.0,3.0,0.0,3.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
35mm 14.3,2.0,1.0,2.0,2.0,4.0,3.0,4.0,2.0,3.0,3.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
35mm 14.4,2.0,2.0,4.0,3.0,5.0,4.0,5.0,8.0,3.0,3.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
35mm 14.5,3.0,3.0,4.0,5.0,10.0,4.0,8.0,7.0,5.0,4.0,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0


In [56]:
transposed_cluster = transposed_cluster.drop(['Peaks_Mass'])


In [57]:
a = transposed_cluster.index
a = list(a)

In [58]:
transposed_cluster= transposed_cluster[(np.char.find(list(a), '2mm', start = 0) >= 0)]


In [59]:
transposed_cluster.index

Index(['2mm 10.0', '2mm 10.1', '2mm 10.2', '2mm 10.3', '2mm 10.4', '2mm 10.5',
       '2mm 10.6', '2mm 10.7', '2mm 10.8', '2mm 10.9', '2mm 11.0', '2mm 11.1',
       '2mm 11.2', '2mm 11.3', '2mm 11.4', '2mm 11.5', '2mm 11.6', '2mm 11.7',
       '2mm 11.8', '2mm 11.9', '2mm 12.0', '2mm 12.1', '2mm 12.2', '2mm 12.3',
       '2mm 12.4', '2mm 12.5', '2mm 12.6', '2mm 12.7', '2mm 12.8', '2mm 12.9',
       '2mm 13.0', '2mm 13.1', '2mm 13.2', '2mm 13.3', '2mm 13.4', '2mm 13.5',
       '2mm 13.6', '2mm 13.7', '2mm 13.8', '2mm 13.9', '2mm 14.0', '2mm 14.1',
       '2mm 14.2', '2mm 14.3', '2mm 14.4', '2mm 14.5'],
      dtype='object')

In [60]:
transposed_cluster

Peaks_Mass,19.0,37.0,55.0,73.0,91.0,109.0,127.0,145.0,163.0,181.0,...,739.0,757.0,775.0,793.0,811.0,829.0,847.0,865.0,883.0,901.0
2mm 10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2mm 10.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2mm 10.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2mm 10.3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2mm 10.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2mm 10.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2mm 10.6,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2mm 10.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2mm 10.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2mm 10.9,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
